In [1]:
import csv
import pandas as pd
import numpy as np
import math
import scipy
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [2]:
appVolDat = pd.read_csv('filingVolumes.csv')
prediction = appVolDat.iloc[1405]
appVolDat = appVolDat.iloc[1272:1392]
appVolDat = appVolDat.reset_index(drop=True)
maximum = max(appVolDat.volume)
appVolDat['vol'] = appVolDat.volume/maximum

trend = pd.read_csv('trend_CostOfPatent.csv').trends.iloc[24:144].reset_index(drop=True)/100
appVolDat['trend1'] = trend#(trend1 - np.mean(trend1)) / np.std(trend1)

trend = pd.read_csv('trend_utilityVDesign.csv').trends.iloc[24:144].reset_index(drop=True)/100
appVolDat['trend2'] = trend#(trend1 - np.mean(trend1)) / np.std(trend1)

trend = pd.read_csv('trend_uspto.csv').trends.iloc[24:144].reset_index(drop=True)/100
appVolDat['trend3'] = trend#(trend1 - np.mean(trend1)) / np.std(trend1)

for i in range(1,13):
    appVolDat['volShift'+str(i)] = appVolDat.vol.shift(i)
appVolDat = appVolDat.iloc[12:]

X_train = appVolDat.iloc[:84]
X_test = appVolDat.iloc[84:]

X_test

,month,volume,vol,trend1,trend2,trend3,volShift1,volShift2,volShift3,volShift4,volShift5,volShift6,volShift7,volShift8,volShift9,volShift10,volShift11,volShift12
96,1/1/2014,40503,0.487512,0.39,0.32,0.68,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603,0.514269,0.496215,0.429605,1.000000,0.500126,0.483889
97,2/1/2014,41112,0.494842,0.71,0.00,0.68,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603,0.514269,0.496215,0.429605,1.000000,0.500126
98,3/1/2014,62356,0.750545,0.55,0.42,0.69,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603,0.514269,0.496215,0.429605,1.000000
99,4/1/2014,44000,0.529604,0.62,0.22,0.69,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603,0.514269,0.496215,0.429605
100,5/1/2014,44872,0.540099,0.65,0.68,0.65,0.529604,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603,0.514269,0.496215
101,6/1/2014,47230,0.568481,0.60,0.00,0.76,0.540099,0.529604,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603,0.514269
102,7/1/2014,43508,0.523682,0.47,0.25,0.69,0.568481,0.540099,0.529604,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603
103,8/1/2014,41927,0.504652,0.33,0.75,0.67,0.523682,0.568481,0.540099,0.529604,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257
104,9/1/2014,47693,0.574054,0.53,0.12,0.69,0.504652,0.523682,0.568481,0.540099,0.529604,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141
105,10/1/2014,45378,0.546190,0.64,0.23,0.69,0.574054,0.504652,0.523682,0.568481,0.540099,0.529604,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267


In [3]:
def func(X,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o):
    return a*(X['volShift1']) + b*(X['volShift2']) + c*(X['volShift3']) + \
d*(X['volShift4']) + e*(X['volShift5']) + f*(X['volShift6']) + \
g*(X['volShift7']) + h*(X['volShift8']) + i*(X['volShift9']) + \
j*(X['volShift10']) + k*(X['volShift11']) + l*(X['volShift12']) + \
m*(X['trend1']) + n*(X['trend2'])# + o*(X['trend3'])

In [4]:
fitParams, fitCovariances = curve_fit(func, X_train, np.array(X_train['vol']))
print(fitParams)

[-0.10607331  0.0169685   0.11445409  0.10258836  0.09689622  0.09120013
  0.12265704 -0.07965364  0.07554496 -0.09648087 -0.05531101  0.68286343
  0.05514836  0.00861776  1.        ]


In [5]:
y_train = func(X_train, fitParams[0], fitParams[1], fitParams[2], fitParams[3], fitParams[4], fitParams[5], fitParams[6], 
            fitParams[7], fitParams[8], fitParams[9], fitParams[10], fitParams[11], fitParams[12], fitParams[13], fitParams[14])
y_test = func(X_test, fitParams[0], fitParams[1], fitParams[2], fitParams[3], fitParams[4], fitParams[5], fitParams[6], 
            fitParams[7], fitParams[8], fitParams[9], fitParams[10], fitParams[11], fitParams[12], fitParams[13], fitParams[14])

In [6]:
X_train.insert(2, 'volPred', y_train)
X_test.insert(2, 'volPred', y_test)

X_train['volPred'] = X_train['volPred'] * maximum
X_test['volPred'] = X_test['volPred'] * maximum

del X_train['vol']
del X_test['vol']

X_test

,month,volume,volPred,trend1,trend2,trend3,volShift1,volShift2,volShift3,volShift4,volShift5,volShift6,volShift7,volShift8,volShift9,volShift10,volShift11,volShift12
96,1/1/2014,40503,36824.250718,0.39,0.32,0.68,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603,0.514269,0.496215,0.429605,1.000000,0.500126,0.483889
97,2/1/2014,41112,42349.502138,0.71,0.00,0.68,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603,0.514269,0.496215,0.429605,1.000000,0.500126
98,3/1/2014,62356,73387.083750,0.55,0.42,0.69,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603,0.514269,0.496215,0.429605,1.000000
99,4/1/2014,44000,37959.251420,0.62,0.22,0.69,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603,0.514269,0.496215,0.429605
100,5/1/2014,44872,44275.377990,0.65,0.68,0.65,0.529604,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603,0.514269,0.496215
101,6/1/2014,47230,46089.925311,0.60,0.00,0.76,0.540099,0.529604,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603,0.514269
102,7/1/2014,43508,44779.113395,0.47,0.25,0.69,0.568481,0.540099,0.529604,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257,0.493603
103,8/1/2014,41927,44160.798647,0.33,0.75,0.67,0.523682,0.568481,0.540099,0.529604,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141,0.514257
104,9/1/2014,47693,48205.850056,0.53,0.12,0.69,0.504652,0.523682,0.568481,0.540099,0.529604,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267,0.534141
105,10/1/2014,45378,48041.757528,0.64,0.23,0.69,0.574054,0.504652,0.523682,0.568481,0.540099,0.529604,0.750545,0.494842,0.487512,0.596261,0.490738,0.541267


In [7]:
print("Num Rows:" , len(X_train), len(X_test))
print("Training Error: ", np.sum(np.absolute(X_train['volPred']-X_train['volume'])) / len(X_train))
print("Testing Error: ", np.sum(np.absolute(X_test['volPred']-X_test['volume'])) / len(X_test))

Num Rows: 84 24
Training Error:  2509.716806189539
Testing Error:  4460.6177503362915
